In [1]:
## Pandas para manejo de Data Frames 
import pandas as pd
## Date Time para manejo de Fechas
import datetime 
## Relacionar script con sharepoint
import requests
from requests.auth import HTTPBasicAuth
import urllib

## Oculto advertencias
import warnings
warnings.filterwarnings("ignore")



#IMPORTACION#

In [2]:
fb=pd.read_excel(r"C:\Users\lkuringhian\Desktop\descargas keepcon\face semana 26.xlsx",sheet_name = "Publicaciones de la Marca" )
ig=pd.read_excel(r"C:\Users\lkuringhian\Desktop\descargas keepcon\instagram semana 26.xlsx",sheet_name = "Publicaciones de la Marca" )

#REEMPLAZO EL NOMBRE DE CUENTA#

In [31]:
### FILTRO EL APOSTROFE EN MC DONALD'S FACEBOOK
Cuenta=fb['Nombre de Cuenta']
x=[]
for i in range(len(fb)):
    x.append(Cuenta[i].replace("\'"," "))
fb['Nombre de Cuenta']=x

### REEMPLAZO NOMBRE DE CUENTA POR NOMBRE DEL PAIS PARA FACEBOOK
y=[]
for i in range(len(fb)):
    y.append(x[i].replace("McDonald s AR","Argentina").replace('McDonald s (BR)','Brasil').replace('McDonald s (CL)','Chile').replace('McDonald s (CR)','Costa Rica').replace('McDonald s (EC)','Ecuador').replace('McDonald s (PA)','Panama').replace('McDonald s (CO)','Colombia').replace('McDonald s (MX)','Mexico').replace('McDonald s (PR)','Puerto Rico').replace('McDonald s (UY)','Uruguay').replace('McDonald s (VE)','Venezuela').replace('McDonald s (PE)','Peru'))
fb['Nombre de Cuenta']=y


### REEMPLAZO NOMBRE DE CUENTA POR NOMBRE DEL PAIS PARA INSTAGRAM

igpais=ig['Nombre de Cuenta']

p=[]
for i in range(len(ig)):
    p.append(igpais[i].replace('mcdonalds_pa','Panama').replace('mcdonalds_uy','Uruguay').replace('mcdonalds_ve','Venezuela').replace('mcdonaldscr','Costa Rica').replace('mcdonalds_ecu','Ecuador').replace('mcdonalds_ar','Argentina').replace('mcdonalds_peru','Peru').replace('mcdonalds_cl','Chile').replace('mcdonaldspr','Puerto Rico').replace('mcdonaldscol','Colombia').replace('mcdonaldsmx','Mexico').replace('mcdonalds_br','Brasil'))
             
ig['Nombre de Cuenta']=p


In [32]:
#Calculo en Numero de la semana basado en el primer dato que tengo
fb['Date'] = pd.to_datetime(fb['Fecha de Publicación'], dayfirst=False)
fb['week_number'] = fb['Fecha de Publicación'].sub(pd.Timestamp('2020-04-27')).dt.days // 7 

ig['Date'] = pd.to_datetime(ig['Fecha de Publicación'], dayfirst=False)
ig['week_number'] = ig['Fecha de Publicación'].sub(pd.Timestamp('2020-04-27')).dt.days // 7 



In [33]:
## Elimino los mensajes vacios en facebook  ### Pueden ser publicaciones Erroneas
fb = fb.dropna(subset=['Mensaje'])
## ELIMINO LOS MENSAJES VACIOS DE INSTAGRAM
ig=ig.dropna(subset=['Mensaje'])

## Elimino las Historias en Instagram
ig=ig[~ig['Permalink'].str.contains("stories")]


Re-Calculo las columnas

In [34]:
# Calculo en Facebook el alcance, total Engagement y Engagement Rate para evitar posibles erroes de la base 

fb['Total Reach']=fb['Alcance Pauta']+fb['Alcance Orgánico']
fb['Total Reactions']=fb['Likes']+fb['Reacciones Sad']+fb['Reacciones Angry']+fb['Reacciones Love']+fb['Reacciones Wow']+fb['Reacciones Haha']
fb['Total Engagement']=fb['Comentarios']+fb['Compartidos']+fb['Total Reactions']
fb['Engagement Rate']=fb['Total Engagement']/fb['Total Reach']

# Calculo lo mismo para Instagram

ig['Total Engagement']=ig['Likes']+ig['Comentarios']
ig['Engagement Rate']=ig['Total Engagement']/ig['Alcance']



In [35]:
fb['day_of_week'] = fb['Fecha de Publicación'].dt.day_name()

ig['day_of_week'] = ig['Fecha de Publicación'].dt.day_name()


In [36]:
### CALCULO EL NUMERO DE SEMANA PARA FACEBOOK 
fe=fb['Fecha de Publicación'].dt.week
fb['Numero de Semana']=fe

## CALCULO EL NUMERO DE SEMANA PARA INSTAGRAM
ige=ig['Fecha de Publicación'].dt.week
ig['Numero de Semana']=ige

#### AVERIGUAR COMO METER EL DIA DEL INICIO DE SEMANA A CADA SEMANA

In [45]:
# Busco publicaciones mal bajadas

errorkeepface = (fb['Total Reach']-fb['Total Reactions'])<0 

### REEMPLAZO TRUE VS A REVISAR Y FALSE POR CORRECTO IDENTIFICANDO LOS ERROES
errorkeepface2 = errorkeepface.map( 
                   {True:'A revisar' ,False:'Correcto'})
print(errorkeepface.value_counts())
print(errorkeepface2.value_counts())

False    598
dtype: int64
Correcto    598
dtype: int64


In [46]:
errorkeepface2

0      Correcto
1      Correcto
2      Correcto
3      Correcto
6      Correcto
         ...   
886    Correcto
887    Correcto
888    Correcto
889    Correcto
890    Correcto
Length: 598, dtype: object

In [38]:
### CREO TABLA CON LOS ERRORES DE KEEPCOM FACEBOOK   ## MEJOR LEGIBILIDAD
fb['Error Reach vs Reactions']=errorkeepface2
fberrores=fb[fb['Error Reach vs Reactions']=='A revisar']
### REEMPLAZAR TRUE Y FALSE POR ALGO CORRECTOS  VS A REVISAR 

In [39]:
### Elimino los erroes de keepcon facebook 

fb=fb[fb['Error Reach vs Reactions']=='Correcto']


In [40]:
## Busco errores en keepcon y verifico

errorkeepig=(ig['Alcance']-ig['Likes'])<0
print(errorkeepig.value_counts())

errorkeepig2 = errorkeepig.map( 
                   {True:'A revisar' ,False:'Correcto'})
print(errorkeepig2.value_counts())



False    61
dtype: int64
Correcto    61
dtype: int64


In [41]:
### CREO TABLA CON LOS ERROES DE INSTAGRAM
ig['Error Reach vs Reactions']=errorkeepig2
igerrores=ig[ig['Error Reach vs Reactions']=='A revisar']
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598 entries, 0 to 890
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Nombre de Cuenta                  598 non-null    object        
 1   Id de Cuenta                      598 non-null    int64         
 2   Tipo de Cuenta                    598 non-null    object        
 3   Id de Publicación                 598 non-null    object        
 4   Fecha de Publicación              598 non-null    datetime64[ns]
 5   Mensaje                           598 non-null    object        
 6   Permalink                         598 non-null    object        
 7   Alcance Pauta                     598 non-null    int64         
 8   Alcance Orgánico                  598 non-null    int64         
 9   Alcance Total                     598 non-null    int64         
 10  Reacciones                        598 non-null    

In [14]:
## Filtro los Errores de Keepcon Instagram 

ig=ig[ig['Error Reach vs Reactions']=='Correcto']


In [15]:
### Filtro facebook por Reach
fb=fb[fb['Total Reach']>0]
### Filtro Instagram por Reach
ig=ig[ig['Alcance']>0]

In [16]:
### Filtro Facebooks por Engagement total
fb=fb[fb['Engagement Total']>0]

### Filtro Instagram segun Engagement Total
ig=ig[ig['Engagement Total']>0]

In [17]:
## ARMO TABLAS CON LOS DATOS DE CADA PAIS DE FACEBOOK

fbargentina=fb[fb['Nombre de Cuenta']=='Argentina']
fbbrasil=fb[fb['Nombre de Cuenta']=='Brasil']
fbperu=fb[fb['Nombre de Cuenta']=='Peru']
fbchile=fb[fb['Nombre de Cuenta']=='Chile']
fbpanama=fb[fb['Nombre de Cuenta']=='Panama']
fbcolombia=fb[fb['Nombre de Cuenta']=='Colombia']
fbvenezuela=fb[fb['Nombre de Cuenta']=='Venezuela']
fbecuador=fb[fb['Nombre de Cuenta']=='Ecuador']
fbpuertorico=fb[fb['Nombre de Cuenta']=='Puerto Rico']
fbcostarica=fb[fb['Nombre de Cuenta']=='Costa Rica']
fbmexico=fb[fb['Nombre de Cuenta']=='Mexico']
fburuguay=fb[fb['Nombre de Cuenta']=='Uruguay']


### ## ARMO TABLAS CON LOS DATOS DE CADA PAIS DE INSTAGRAM

igargentina=ig[ig['Nombre de Cuenta']=='Argentina']
igbrasil=ig[ig['Nombre de Cuenta']=='Brasil']
igperu=ig[ig['Nombre de Cuenta']=='Peru']
igchile=ig[ig['Nombre de Cuenta']=='Chile']
igpanama=ig[ig['Nombre de Cuenta']=='Panama']
igcolombia=ig[ig['Nombre de Cuenta']=='Colombia']
igvenezuela=ig[ig['Nombre de Cuenta']=='Venezuela']
igecuador=ig[ig['Nombre de Cuenta']=='Ecuador']
igpuertorico=ig[ig['Nombre de Cuenta']=='Puerto Rico']
igcostarica=ig[ig['Nombre de Cuenta']=='Costa Rica']
igmexico=ig[ig['Nombre de Cuenta']=='Mexico']
iguruguay=ig[ig['Nombre de Cuenta']=='Uruguay']


In [18]:
## ELIMINO EL PERCENTIL 20 CON RESPECTO AL ALCANCE TOTAL

# Armo percentiles con respecto al alcance total para cada uno de los paises de facebook
fbargentina["Percentil"]=fbargentina['Alcance Total'].rank(pct=True)
fbbrasil["Percentil"]=fbbrasil['Alcance Total'].rank(pct=True)
fbperu["Percentil"]=fbperu['Alcance Total'].rank(pct=True)
fbchile["Percentil"]=fbchile['Alcance Total'].rank(pct=True)
fbpanama["Percentil"]=fbpanama['Alcance Total'].rank(pct=True)
fbcolombia["Percentil"]=fbcolombia['Alcance Total'].rank(pct=True)
fbvenezuela["Percentil"]=fbvenezuela['Alcance Total'].rank(pct=True)
fbecuador["Percentil"]=fbecuador['Alcance Total'].rank(pct=True)
fbpuertorico["Percentil"]=fbpuertorico['Alcance Total'].rank(pct=True)
fbcostarica["Percentil"]=fbcostarica['Alcance Total'].rank(pct=True)
fbmexico["Percentil"]=fbmexico['Alcance Total'].rank(pct=True)
fburuguay["Percentil"]=fburuguay['Alcance Total'].rank(pct=True)




In [19]:
# Armo percentiles con respecto al alcance total para cada uno de los paises de instagram

igargentina["Percentil"]=igargentina['Alcance'].rank(pct=True)
igbrasil["Percentil"]=igbrasil['Alcance'].rank(pct=True)
igperu["Percentil"]=igperu['Alcance'].rank(pct=True)
igchile["Percentil"]=igchile['Alcance'].rank(pct=True)
igpanama["Percentil"]=igpanama['Alcance'].rank(pct=True)
igcolombia["Percentil"]=igcolombia['Alcance'].rank(pct=True)
igvenezuela["Percentil"]=igvenezuela['Alcance'].rank(pct=True)
igecuador["Percentil"]=igecuador['Alcance'].rank(pct=True)
igpuertorico["Percentil"]=igpuertorico['Alcance'].rank(pct=True)
igcostarica["Percentil"]=igcostarica['Alcance'].rank(pct=True)
igmexico["Percentil"]=igmexico['Alcance'].rank(pct=True)
iguruguay["Percentil"]=iguruguay['Alcance'].rank(pct=True)


In [20]:
## FILTRO EL PERCENTIL 20 DE CADA PAIS DE FACEBOOK

fbargentina=fbargentina[fbargentina["Percentil"]>=0.2]
fbbrasil=fbbrasil[fbbrasil["Percentil"]>=0.2]
fbperu=fbperu[fbperu["Percentil"]>=0.2]
fbchile=fbchile[fbchile["Percentil"]>=0.2]
fbpanama=fbpanama[fbpanama["Percentil"]>=0.2]
fbcolombia=fbcolombia[fbcolombia["Percentil"]>=0.2]
fbvenezuela=fbvenezuela[fbvenezuela["Percentil"]>=0.2]
fbecuador=fbecuador[fbecuador["Percentil"]>=0.2]
fbpuertorico=fbpuertorico[fbpuertorico["Percentil"]>=0.2]
fbcostarica=fbcostarica[fbcostarica["Percentil"]>=0.2]
fbmexico=fbmexico[fbmexico["Percentil"]>=0.2]
fburuguay=fburuguay[fburuguay["Percentil"]>=0.2]

In [21]:
topfbargentina=fbargentina.nlargest(3,'Engagement Total')
topfbbrasil=fbbrasil.nlargest(3,'Engagement Total')
topfbperu=fbperu.nlargest(3,'Engagement Total')
topfbchile=fbchile.nlargest(3,'Engagement Total')
topfbpanama=fbpanama.nlargest(3,'Engagement Total')
topfbcolombia=fbcolombia.nlargest(3,'Engagement Total')
topfbvenezuela=fbvenezuela.nlargest(3,'Engagement Total')
topfbecuador=fbecuador.nlargest(3,'Engagement Total')
topfbpuertorico=fbpuertorico.nlargest(3,'Engagement Total')
topfbcostarica=fbcostarica.nlargest(3,'Engagement Total')
topfbmexico=fbmexico.nlargest(3,'Engagement Total')
topfburuguay=fburuguay.nlargest(3,'Engagement Total')

topigargentina=igargentina.nlargest(3,'Engagement Total')
topigbrasil=igbrasil.nlargest(3,'Engagement Total')
topigperu=igperu.nlargest(3,'Engagement Total')
topigchile=igchile.nlargest(3,'Engagement Total')
topigpanama=igpanama.nlargest(3,'Engagement Total')
topigcolombia=igcolombia.nlargest(3,'Engagement Total')
topigvenezuela=igvenezuela.nlargest(3,'Engagement Total')
topigecuador=igecuador.nlargest(3,'Engagement Total')
topigpuertorico=igpuertorico.nlargest(3,'Engagement Total')
topigcostarica=igcostarica.nlargest(3,'Engagement Total')
topigmexico=igmexico.nlargest(3,'Engagement Total')
topiguruguay=iguruguay.nlargest(3,'Engagement Total')


In [22]:
topfbarg2=topfbargentina[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbbrasil2=topfbbrasil[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbperu2=topfbperu[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbchile2=topfbchile[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbpanama2=topfbpanama[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbcolombia2=topfbcolombia[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbvenezuela2=topfbvenezuela[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbecuador2=topfbecuador[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbpuertorico2=topfbpuertorico[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbcostarica2=topfbcostarica[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfbmexico2=topfbmexico[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]
topfburuguay2=topfburuguay[['Nombre de Cuenta','Permalink','Engagement Total','Alcance Total']]


topigargentina2=topigargentina[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigbrasil2=topigbrasil[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigperu2=topigperu[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigchile2=topigchile[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigpanama2=topigpanama[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigcolombia2=topigcolombia[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigvenezuela2=topigvenezuela[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigecuador2=topigecuador[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigpuertorico2=topigpuertorico[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigcostarica2=topigcostarica[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topigmexico2=topigmexico[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topiguruguay2=topiguruguay[['Nombre de Cuenta','Permalink','Engagement Total','Alcance']]
topfbarg2

,Nombre de Cuenta,Permalink,Engagement Total,Alcance Total
4761,Argentina,http://www.facebook.com/79667539019/posts/6969...,9728,7736213
13531,Argentina,http://www.facebook.com/79667539019/posts/6969...,9728,7736213
4262,Argentina,http://www.facebook.com/79667539019/posts/7040...,8414,2496638


In [23]:
## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
fbtop=[topfbarg2,topfbbrasil2,topfbperu2,topfbchile2,topfbpanama2,topfbcolombia2,topfbvenezuela2,topfbecuador2,topfbpuertorico2,topfbcostarica2,topfbmexico2,topfburuguay2]
topfb = pd.concat(fbtop)



## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
igtop=[topigargentina2,topigbrasil2,topigperu2,topigchile2,topigpanama2,topigcolombia2,topigvenezuela2,topigecuador2,topigpuertorico2,topigcostarica2,topigmexico2,topiguruguay2]
topig = pd.concat(igtop)

In [24]:
a='Argentina'
b='Brasil'
c='Colombia'
d='Chile'
e='Panama'
f='Venezuela'
g='Ecuador'
h='Puerto Rico'
i='Costa Rica'
j='Mexico'
k='Uruguay'
l='Peru'

In [25]:
fbargentinagroup=fbargentina.groupby('week_number').agg({'Engagement Rate':['mean']})
fbargentinagroup['Pais']=a

fbbrasilgroup=fbbrasil.groupby('week_number').agg({'Engagement Rate':['mean']})
fbbrasilgroup['Pais']=b

fbcolombiagroup=fbcolombia.groupby('week_number').agg({'Engagement Rate':['mean']})
fbcolombiagroup['Pais']=c

fbchilegroup=fbchile.groupby('week_number').agg({'Engagement Rate':['mean']})
fbchilegroup['Pais']=d

fbpanamagroup=fbpanama.groupby('week_number').agg({'Engagement Rate':['mean']})
fbpanamagroup['Pais']=e

fbvenezuelagroup=fbvenezuela.groupby('week_number').agg({'Engagement Rate':['mean']})
fbvenezuelagroup['Pais']=f

fbecuadorgroup=fbecuador.groupby('week_number').agg({'Engagement Rate':['mean']})
fbecuadorgroup['Pais']=g

fbpuertoricogroup=fbpuertorico.groupby('week_number').agg({'Engagement Rate':['mean']})
fbpuertoricogroup['Pais']=h

fbcostaricagroup=fbcostarica.groupby('week_number').agg({'Engagement Rate':['mean']})
fbcostaricagroup['Pais']=i

fbmexicogroup=fbmexico.groupby('week_number').agg({'Engagement Rate':['mean']})
fbmexicogroup['Pais']=j

fburuguaygroup=fburuguay.groupby('week_number').agg({'Engagement Rate':['mean']})
fburuguaygroup['Pais']=k

fbperugroup=fbperu.groupby('week_number').agg({'Engagement Rate':['mean']})
fbperugroup['Pais']=l

In [26]:
igargentinagroup=igargentina.groupby('week_number').agg({'Engagement Rate':['mean']})
igargentinagroup['Pais']=a

igbrasilgroup=igbrasil.groupby('week_number').agg({'Engagement Rate':['mean']})
igbrasilgroup['Pais']=b

igcolombiagroup=igcolombia.groupby('week_number').agg({'Engagement Rate':['mean']})
igcolombiagroup['Pais']=c

igchilegroup=igchile.groupby('week_number').agg({'Engagement Rate':['mean']})
igchilegroup['Pais']=d

igpanamagroup=igpanama.groupby('week_number').agg({'Engagement Rate':['mean']})
igpanamagroup['Pais']=e

igvenezuelagroup=igvenezuela.groupby('week_number').agg({'Engagement Rate':['mean']})
igvenezuelagroup['Pais']=f

igecuadorgroup=igecuador.groupby('week_number').agg({'Engagement Rate':['mean']})
igecuadorgroup['Pais']=g

igpuertoricogroup=igpuertorico.groupby('week_number').agg({'Engagement Rate':['mean']})
igpuertoricogroup['Pais']=h

igcostaricagroup=igcostarica.groupby('week_number').agg({'Engagement Rate':['mean']})
igcostaricagroup['Pais']=i

igmexicogroup=igmexico.groupby('week_number').agg({'Engagement Rate':['mean']})
igmexicogroup['Pais']=j

iguruguaygroup=iguruguay.groupby('week_number').agg({'Engagement Rate':['mean']})
iguruguaygroup['Pais']=k

igperugroup=igperu.groupby('week_number').agg({'Engagement Rate':['mean']})
igperugroup['Pais']=l

In [27]:
fbargentinagroup['Benchmark Movil']=fbargentinagroup['Engagement Rate'].rolling(13).mean()

fbbrasilgroup['Benchmark Movil']=fbbrasilgroup['Engagement Rate'].rolling(13).mean()

fbcolombiagroup['Benchmark Movil']=fbcolombiagroup['Engagement Rate'].rolling(13).mean()

fbchilegroup['Benchmark Movil']=fbchilegroup['Engagement Rate'].rolling(13).mean()

fbpanamagroup['Benchmark Movil']=fbpanamagroup['Engagement Rate'].rolling(13).mean()

fbvenezuelagroup['Benchmark Movil']=fbvenezuelagroup['Engagement Rate'].rolling(13).mean()

fbecuadorgroup['Benchmark Movil']=fbecuadorgroup['Engagement Rate'].rolling(13).mean()

fbpuertoricogroup['Benchmark Movil']=fbpuertoricogroup['Engagement Rate'].rolling(13).mean()

fbcostaricagroup['Benchmark Movil']=fbcostaricagroup['Engagement Rate'].rolling(13).mean()

fbmexicogroup['Benchmark Movil']=fbmexicogroup['Engagement Rate'].rolling(13).mean()

fburuguaygroup['Benchmark Movil']=fburuguaygroup['Engagement Rate'].rolling(13).mean()

fbperugroup['Benchmark Movil']=fbperugroup['Engagement Rate'].rolling(13).mean()

In [28]:
igargentinagroup['Benchmark Movil']=igargentinagroup['Engagement Rate'].rolling(13).mean()

igbrasilgroup['Benchmark Movil']=igbrasilgroup['Engagement Rate'].rolling(13).mean()

igcolombiagroup['Benchmark Movil']=igcolombiagroup['Engagement Rate'].rolling(13).mean()

igchilegroup['Benchmark Movil']=igchilegroup['Engagement Rate'].rolling(13).mean()

igpanamagroup['Benchmark Movil']=igpanamagroup['Engagement Rate'].rolling(13).mean()

igvenezuelagroup['Benchmark Movil']=igvenezuelagroup['Engagement Rate'].rolling(13).mean()

igecuadorgroup['Benchmark Movil']=igecuadorgroup['Engagement Rate'].rolling(13).mean()

igpuertoricogroup['Benchmark Movil']=igpuertoricogroup['Engagement Rate'].rolling(13).mean()

igcostaricagroup['Benchmark Movil']=igcostaricagroup['Engagement Rate'].rolling(13).mean()

igmexicogroup['Benchmark Movil']=igmexicogroup['Engagement Rate'].rolling(13).mean()

iguruguaygroup['Benchmark Movil']=iguruguaygroup['Engagement Rate'].rolling(13).mean()

igperugroup['Benchmark Movil']=igperugroup['Engagement Rate'].rolling(13).mean()

In [29]:
## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
facebookmerge=[fbargentinagroup,fbbrasilgroup,fbperugroup,fbchilegroup,fbpanamagroup,fbcolombiagroup,fbvenezuelagroup,fbecuadorgroup,fbpuertoricogroup,fbcostaricagroup,fbmexicogroup,fburuguaygroup]
fbench = pd.concat(facebookmerge)



## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
instagrammerge=[igargentinagroup,igbrasilgroup,igperugroup,igchilegroup,igpanamagroup,igcolombiagroup,igvenezuelagroup,igecuadorgroup,igpuertoricogroup,igcostaricagroup,igmexicogroup,iguruguaygroup]
igbench = pd.concat(instagrammerge)


In [30]:
## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
facebookmerge2=[fbargentina,fbbrasil,fbperu,fbchile,fbpanama,fbcolombia,fbvenezuela,fbecuador,fbpuertorico,fbcostarica,fbmexico,fburuguay]
fb = pd.concat(facebookmerge2)



## HAGO MERGE DE LOS DATA FRAMES DE FACEBOOK
instagrammerge2=[igargentina,igbrasil,igperu,igchile,igpanama,igcolombia,igvenezuela,igecuador,igpuertorico,igcostarica,igmexico,iguruguay]
ig = pd.concat(instagrammerge2)


In [31]:
###### CALCULO BENCHMARK 

#############  UNA VEZ QUE ESTA TODO LIMPIO Y ACOMODADO NOS ENFOCAMOS EN LOS CALCULOS Y OBTENER LA INFORMACION UTIL

In [32]:
fbbenchmark=fb.groupby('Nombre de Cuenta').agg({'Total Engagement':['mean','count'],'Total Reach':['mean'],'Numero de Semana':['mean']})
igbenchmark=ig.groupby('Nombre de Cuenta').agg({'Total Engagement':['mean','count'],'Alcance':['mean'],'Numero de Semana':['mean']})

fbbenchindex=fbbenchmark.reset_index()
fbbenchindex.columns=['Nombre de Cuenta','Total Engagement','Total Engagement Count','Total Reach Mean','Numero de Semana']
igbenchmarkindex=igbenchmark.reset_index()
igbenchmarkindex.columns=['Nombre de Cuenta','Total Engagement','Total Engagement Count','Total Reach Mean','Numero de Semana']

In [33]:
## CREO EXCEL PARA ACTUALIZAR EL KPI 1
fbkpi=fb.groupby('Nombre de Cuenta').agg({'Engagement Rate':['count','mean']})
igkpi=ig.groupby('Nombre de Cuenta').agg({'Engagement Rate':['count','mean']})

In [34]:
## EXPORTO LOS ERRORES TANTO DE FACEBOOK COMO DE INSTAGRAM A UN EXCEL

writer = pd.ExcelWriter('ErroresKeeerrpcon.xlsx', engine='xlsxwriter')

fberrores.to_excel(writer, sheet_name='Errores Facebook')

igerrores.to_excel(writer, sheet_name='Errores Instagram')

writer.save()

In [35]:
## EXPORT LOS BENCHMARK CALCULADOS TANTO PARA FACEBOOK COMO INSTAGRAM
writer2 = pd.ExcelWriter('Benchmark.xlsx', engine='xlsxwriter')

fbench.to_excel(writer2, sheet_name='Benchmark Facebook')

igbench.to_excel(writer2, sheet_name='Benchmark Instagram')

writer2.save()

In [91]:
## EXPORTO LOS DATOS YA FILTRADOS A UN EXCEL
writer3 = pd.ExcelWriter('Semana 26.xlsx', engine='xlsxwriter')

fb.to_excel(writer3, sheet_name='Facebook')

ig.to_excel(writer3, sheet_name='Instagram')

writer3.save()

In [92]:
## EXPORTO DATOS PARA LA ACTUALIZACION DEL KPI 1
writer4 = pd.ExcelWriter('Actualizacion Kpi 1 .xlsx', engine='xlsxwriter')

fbkpi.to_excel(writer4, sheet_name='Facebook')

igkpi.to_excel(writer4, sheet_name='Instagram')

writer4.save()

In [93]:
## EXPORTO DATOS PARA LA ACTUALIZACION DEL KPI 1
writer5 = pd.ExcelWriter('Actualizacion kpi 2.xlsx', engine='xlsxwriter')

topfb.to_excel(writer5, sheet_name='Facebook')

topig.to_excel(writer5, sheet_name='Instagram')

writer5.save()

In [94]:
## ARMO TABLAS CON LOS DATOS DE CADA PAIS DE FACEBOOK

fbargentina=fb[fb['Nombre de Cuenta']=='Argentina']
fbbrasil=fb[fb['Nombre de Cuenta']=='Brasil']
fbperu=fb[fb['Nombre de Cuenta']=='Peru']
fbchile=fb[fb['Nombre de Cuenta']=='Chile']
fbpanama=fb[fb['Nombre de Cuenta']=='Panama']
fbcolombia=fb[fb['Nombre de Cuenta']=='Colombia']
fbvenezuela=fb[fb['Nombre de Cuenta']=='Venezuela']
fbecuador=fb[fb['Nombre de Cuenta']=='Ecuador']
fbpuertorico=fb[fb['Nombre de Cuenta']=='Puerto Rico']
fbcostarica=fb[fb['Nombre de Cuenta']=='Costa Rica']
fbmexico=fb[fb['Nombre de Cuenta']=='Mexico']
fburuguay=fb[fb['Nombre de Cuenta']=='Uruguay']


## CONTINUARA... #

In [2]:
paises=('Argentina','Brasil')

SyntaxError: invalid syntax (<ipython-input-9-191681f599fa>, line 3)